<a href="https://colab.research.google.com/github/vinay-jose/rag-nbs/blob/main/Full_Text_to_LLM_using_Unstructured_io_%2B_Ollama.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!sudo apt-get install poppler-utils
!sudo apt-get install tesseract-ocr

In [ ]:
!pip install -Uqq "unstructured[all-docs]" pytesseract

In [ ]:
from unstructured.partition.pdf import partition_pdf

In [ ]:
elements = partition_pdf(filename=fname,
                         infer_table_structure=True,
                         strategy='hi_res', # https://docs.unstructured.io/open-source/concepts/partitioning-strategies
                         hi_res_model_name="yolox_quantized" # https://docs.unstructured.io/open-source/concepts/models - yolox, yolox_quantized, detectron2_onnx
           )

In [ ]:
documents = []
page = ""
tmp = elements[0].metadata.page_number
for el in elements:
  if el.category == "Table":
    page += el.metadata.text_as_html
  elif el.category == "Header":
    page += f"<h1>{el.text}</h1>"
  elif el.category == "Title":
    page += f"<h2>{el.text}</h2>"
  else:
    page += el.text

  page += "\n"

  if el.metadata.page_number != tmp:
    docs.append(page)
    page = ""
    tmp = el.metadata.page_number

docs.append(page)

len(documents)

In [ ]:
print(documents[0])

In [ ]:
prompt_template = f"""
You are an AI assistant. Given the user query: "{query}", and the following list of relevant documents:

{documents}

Generate a coherent and informative response that addresses the user's query using the information from the documents.
"""